In [1]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.6.0


In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: mlopsdev
Azure region: centralus
Subscription id: c46a9435-c957-4e6c-a0f4-b9a597984773
Resource group: mlops


In [3]:
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.datastore import Datastore
from azureml.data.data_reference import DataReference
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.pipeline.steps import PythonScriptStep
from azureml.core.runconfig import CondaDependencies, RunConfiguration
from azureml.core.compute_target import ComputeTargetException

In [4]:
import os

# create directory for model
model_dir = 'models'
if not os.path.isdir(model_dir):
    os.mkdir(model_dir)

In [5]:
import urllib.request

def download_model(model_name):
    # downloaded models from https://pytorch.org/tutorials/advanced/neural_style_tutorial.html are kept here
    url="https://pipelinedata.blob.core.windows.net/styletransfer/saved_models/" + model_name
    local_path = os.path.join(model_dir, model_name)
    urllib.request.urlretrieve(url, local_path)

In [6]:

from azureml.core.model import Model
mosaic_model = None
candy_model = None

models = Model.list(workspace=ws, tags=['scenario'])
for m in models:
    print("Name:", m.name,"\tVersion:", m.version, "\tDescription:", m.description, m.tags)
    if m.name == 'mosaic' and mosaic_model is None:
        mosaic_model = m
    elif m.name == 'candy' and candy_model is None:
        candy_model = m

if mosaic_model is None:
    print('Mosaic model does not exist, registering it')
    download_model('mosaic.pth')
    mosaic_model = Model.register(model_path = os.path.join(model_dir, "mosaic.pth"),
                       model_name = "mosaic",
                       tags = {'type': "mosaic", 'scenario': "Style transfer using batch inference"},
                       description = "Style transfer - Mosaic",
                       workspace = ws)
else:
    print('Reusing existing mosaic model')
    

if candy_model is None:
    print('Candy model does not exist, registering it')
    download_model('candy.pth')
    candy_model = Model.register(model_path = os.path.join(model_dir, "candy.pth"),
                       model_name = "candy",
                       tags = {'type': "candy", 'scenario': "Style transfer using batch inference"},
                       description = "Style transfer - Candy",
                       workspace = ws)
else:
    print('Reusing existing candy model')

Name: candy 	Version: 1 	Description: Style transfer - Candy {'type': 'candy', 'scenario': 'Style transfer using batch inference'}
Name: mosaic 	Version: 1 	Description: Style transfer - Mosaic {'type': 'mosaic', 'scenario': 'Style transfer using batch inference'}
Reusing existing mosaic model
Reusing existing candy model


In [7]:
# AmlCompute
cpu_cluster_name = "cpu-cluster"
try:
    cpu_cluster = AmlCompute(ws, cpu_cluster_name)
    print("found existing cluster.")
except ComputeTargetException:
    print("creating new cluster")
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_v2",
                                                                    max_nodes = 1)

    # create the cluster
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, provisioning_config)
    cpu_cluster.wait_for_completion(show_output=True)
    
# AmlCompute
gpu_cluster_name = "gpu-cluster"
try:
    gpu_cluster = AmlCompute(ws, gpu_cluster_name)
    print("found existing cluster.")
except ComputeTargetException:
    print("creating new cluster")
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_NC6",
                                                                max_nodes = 3)

    # create the cluster
    gpu_cluster = ComputeTarget.create(ws, gpu_cluster_name, provisioning_config)
    gpu_cluster.wait_for_completion(show_output=True)

found existing cluster.
creating new cluster
Creating
AmlCompute wait for completion finished

Terminal state of "Failed" has been reached

Provisioning errors: [{'error': {'code': 'VmSizeNotSupported', 'message': 'STANDARD_NC6 is not supported in region centralus. Please choose a different VM size.', 'details': []}}]



In [ ]:

scripts_folder = "scripts"

In [11]:
process_video_script_file = "process_video.py"

# peek at contents
with open(os.path.join(scripts_folder, process_video_script_file)) as process_video_file:
    print(process_video_file.read())

import argparse
import glob
import os
import subprocess

parser = argparse.ArgumentParser(description="Process input video")
parser.add_argument('--input_video', required=True)
parser.add_argument('--output_audio', required=True)
parser.add_argument('--output_images', required=True)

args = parser.parse_args()

os.makedirs(args.output_audio, exist_ok=True)
os.makedirs(args.output_images, exist_ok=True)

subprocess.run("ffmpeg -i {} {}/video.aac".format(args.input_video, args.output_audio),
               shell=True,
               check=True)

subprocess.run("ffmpeg -i {} {}/%05d_video.jpg -hide_banner".format(args.input_video, args.output_images),
               shell=True,
               check=True)


In [12]:
stitch_video_script_file = "stitch_video.py"

# peek at contents
with open(os.path.join(scripts_folder, stitch_video_script_file)) as stitch_video_file:
    print(stitch_video_file.read())

import argparse
import os
import subprocess

parser = argparse.ArgumentParser(description="Process input video")
parser.add_argument('--images_dir', required=True)
parser.add_argument('--input_audio', required=True)
parser.add_argument('--output_dir', required=True)

args = parser.parse_args()

os.makedirs(args.output_dir, exist_ok=True)

subprocess.run("ffmpeg -framerate 30 -i {}/%05d_video.jpg -c:v libx264 -profile:v high -crf 20 -pix_fmt yuv420p "
               "-y {}/video_without_audio.mp4"
               .format(args.images_dir, args.output_dir),
               shell=True, check=True)

subprocess.run("ffmpeg -i {}/video_without_audio.mp4 -i {}/video.aac -map 0:0 -map 1:0 -vcodec "
               "copy -acodec copy -y {}/video_with_audio.mp4"
               .format(args.output_dir, args.input_audio, args.output_dir),
               shell=True, check=True)


In [13]:

# datastore for input video
account_name = "pipelinedata"
video_ds = Datastore.register_azure_blob_container(ws, "videos", "sample-videos",
                                            account_name=account_name, overwrite=True)

# the default blob store attached to a workspace
default_datastore = ws.get_default_datastore()

In [14]:
# datastore for input video
account_name = "pipelinedata"
video_ds = Datastore.register_azure_blob_container(ws, "videos", "sample-videos",
                                            account_name=account_name, overwrite=True)

# the default blob store attached to a workspace
default_datastore = ws.get_default_datastore()

In [15]:
video_name=os.getenv("STYLE_TRANSFER_VIDEO_NAME", "orangutan.mp4") 
orangutan_video = DataReference(datastore=video_ds,
                            data_reference_name="video",
                            path_on_datastore=video_name, mode="download")

In [16]:
cd = CondaDependencies()

cd.add_channel("conda-forge")
cd.add_conda_package("ffmpeg")

# Runconfig
amlcompute_run_config = RunConfiguration(conda_dependencies=cd)
amlcompute_run_config.environment.docker.base_image = "pytorch/pytorch"
amlcompute_run_config.environment.spark.precache_packages = False

In [17]:
ffmpeg_audio = PipelineData(name="ffmpeg_audio", datastore=default_datastore)
processed_images = PipelineData(name="processed_images", datastore=default_datastore)
output_video = PipelineData(name="output_video", datastore=default_datastore)

ffmpeg_images_ds_name = "ffmpeg_images_data"
ffmpeg_images = PipelineData(name="ffmpeg_images", datastore=default_datastore)
ffmpeg_images_file_dataset = ffmpeg_images.as_dataset()
ffmpeg_images_named_file_dataset = ffmpeg_images_file_dataset.as_named_input(ffmpeg_images_ds_name)

In [18]:
from azureml.pipeline.core.graph import PipelineParameter
# create a parameter for style (one of "candy", "mosaic") to transfer the images to
style_param = PipelineParameter(name="style", default_value="mosaic")

In [19]:

split_video_step = PythonScriptStep(
    name="split video",
    script_name="process_video.py",
    arguments=["--input_video", orangutan_video,
               "--output_audio", ffmpeg_audio,
               "--output_images", ffmpeg_images,
              ],
    compute_target=cpu_cluster,
    inputs=[orangutan_video],
    outputs=[ffmpeg_images, ffmpeg_audio],
    runconfig=amlcompute_run_config,
    source_directory=scripts_folder
)

stitch_video_step = PythonScriptStep(
    name="stitch",
    script_name="stitch_video.py",
    arguments=["--images_dir", processed_images, 
               "--input_audio", ffmpeg_audio, 
               "--output_dir", output_video],
    compute_target=cpu_cluster,
    inputs=[processed_images, ffmpeg_audio],
    outputs=[output_video],
    runconfig=amlcompute_run_config,
    source_directory=scripts_folder
)

In [20]:
from azureml.core import Environment
from azureml.core.runconfig import DEFAULT_GPU_IMAGE

parallel_cd = CondaDependencies()

parallel_cd.add_channel("pytorch")
parallel_cd.add_conda_package("pytorch")
parallel_cd.add_conda_package("torchvision")

styleenvironment = Environment(name="styleenvironment")
styleenvironment.python.conda_dependencies=parallel_cd
styleenvironment.docker.base_image = DEFAULT_GPU_IMAGE

In [24]:
from azureml.contrib.pipeline.steps import ParallelRunConfig

parallel_run_config = ParallelRunConfig(
                    environment=styleenvironment,
                    entry_script='transform.py',
                    output_action='summary_only',
                    mini_batch_size="1",
                    error_threshold=1,
                    source_directory=scripts_folder,
                    compute_target=cpu_cluster, 
                    node_count=3)

ValueError: Parameter node_count must be between 1 and max_nodes 1

In [22]:
from azureml.contrib.pipeline.steps import ParallelRunStep
from datetime import datetime

parallel_step_name = 'styletransfer-' + datetime.now().strftime('%Y%m%d%H%M')

distributed_style_transfer_step = ParallelRunStep(
    name=parallel_step_name,
    inputs=[ffmpeg_images_named_file_dataset], # Input file share/blob container/file dataset
    output=processed_images,  # Output file share/blob container
    models=[mosaic_model, candy_model],
    tags = {'scenario': "batch inference", 'type': "demo"},
    properties = {'area': "style transfer"},
    arguments=["--style", style_param],
    parallel_run_config=parallel_run_config,
    allow_reuse=True #[optional - default value True]
)

In [23]:
pipeline = Pipeline(workspace=ws, steps=[stitch_video_step])

pipeline.validate()

TypeError: create_module_def() got an unexpected keyword argument 'arguments'

In [ ]:
# submit the pipeline and provide values for the PipelineParameters used in the pipeline
pipeline_run = Experiment(ws, 'styletransfer_parallel_mosaic').submit(pipeline)

In [ ]:
# Track pipeline run progress
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

In [ ]:
pipeline_run.wait_for_completion()

In [ ]:
def download_video(run, target_dir=None):
    stitch_run = run.find_step_run("stitch")[0]
    port_data = stitch_run.get_output_data("output_video")
    port_data.download(target_dir, show_progress=True)

In [ ]:
pipeline_run.wait_for_completion()
download_video(pipeline_run, "output_video_mosaic")

In [ ]:
pipeline_name = "style-transfer-batch-inference"
print(pipeline_name)

published_pipeline = pipeline.publish(
    name=pipeline_name, 
    description=pipeline_name)
print("Newly published pipeline id: {}".format(published_pipeline.id))

In [ ]:
from azureml.pipeline.core import PublishedPipeline

# You could retrieve all pipelines that are published, or 
# just get the published pipeline object that you have the ID for.

# Get all published pipeline objects in the workspace
all_pub_pipelines = PublishedPipeline.list(ws)

# We will iterate through the list of published pipelines and 
# use the last ID in the list for Schelue operations: 
print("Published pipelines found in the workspace:")
for pub_pipeline in all_pub_pipelines:
    print("Name:", pub_pipeline.name,"\tDescription:", pub_pipeline.description, "\tId:", pub_pipeline.id, "\tStatus:", pub_pipeline.status)
    if(pub_pipeline.name == pipeline_name):
        published_pipeline = pub_pipeline

print("Published pipeline id: {}".format(published_pipeline.id))

In [ ]:
from azureml.core.authentication import InteractiveLoginAuthentication
import requests

auth = InteractiveLoginAuthentication()
aad_token = auth.get_authentication_header()

In [ ]:
rest_endpoint = published_pipeline.endpoint
print("Pipeline REST endpoing: {}".format(rest_endpoint))

In [ ]:
experiment_name = 'styletransfer_parallel_candy'
response = requests.post(rest_endpoint, 
                         headers=aad_token,
                         json={"ExperimentName": experiment_name,
                               "ParameterAssignments": {"style": "candy", "aml_node_count": 2}})
run_id = response.json()["Id"]

from azureml.pipeline.core.run import PipelineRun
published_pipeline_run_candy = PipelineRun(ws.experiments[experiment_name], run_id)

RunDetails(published_pipeline_run_candy).show()

In [ ]:
published_pipeline_run_candy.wait_for_completion()

In [ ]:
download_video(published_pipeline_run_candy, target_dir="output_video_candy"